### Retrieval

**RAG의 5단계**
1. **Document Loader**: 문서를 불러오고
2. **Document Transformer**: 문서를 쪼개고
3. **Embedding**: 텍스트를 숫자로 바꾸고
4. **Vector Store**: 저장소에 넣고
5. **Retrieval** : 검색해서 LLM에 전달합니다.

In [ ]:
%pip install langchain-community pypdf faiss-cpu sentence-transformers

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### Document Loader(문서 불러오기)

In [ ]:
%pip install bs4

In [ ]:
from langchain_community.document_loaders import WebBaseLoader # 웹페이지 URL에서 텍스트를 긁어오는 도구

url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8"

# 로더 인스턴스 생성
loader = WebBaseLoader(url)

# 해당 URL에 접속하여 HTML 파싱, 텍스트만 추출하여 Document 객체 리스트로 반환
documents = loader.load()

print(len(documents))
print(documents[0].metadata)

# 본문 내용 확인
print(documents[0].page_content[:500])

In [ ]:
from langchain_community.document_loaders import PyPDFLoader # PDF 파일을 로드하여 텍스트로 변환하는 도구

# 로더 인스턴스 생성 (파일 경로 지정)
loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")

# 문서 로드 실행 : PDF 각 페이지를 하나의 Document 객체로 변환하여 리스트로 반환
documents = loader.load()

print(len(documents))
print(documents[0].metadata)
print(documents[3].page_content)    # 4번째 페이지(인덱스 3)의 본문 내용 

### Embedding Model(임베딩: 텍스트를 숫자로)

In [ ]:
from langchain_openai import OpenAIEmbeddings
import pandas as pd

# 임베딩 모델 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

text="The quick brown fox jumps over the lazy dog."
vector = embeddings.embed_query(text)   # 하나의 문자열을 벡터로 변환

print(len(vector))
print(pd.Series(vector).head())

In [ ]:
# 문서 내용만 추출
docs = [document.page_content for document in documents]
print(len(docs))

# embed_documents() : 문자열 리스트를 받아서, 각각을 벡터로 변환한 뒤 '벡터 리스트'를 반환
vects = embeddings.embed_documents(docs)

print(len(vects))
print(len(vects[0]))
pd.DataFrame(vects)

### vector store(FAISS)


In [ ]:
# FAISS 
from langchain_community.vectorstores import FAISS

# 벡터 저장소 생성
# 이 함수 내부에서 'documents'의 텍스트를 'embeddings'모델로 벡터화하고, FAISS 인덱스를 만들어 저장한다.
vector_store = FAISS.from_documents(documents, embeddings)

print(vector_store)

# 유사도 검색
query="Tom Sawyer"

# similarity_search() :  질문(query)와 가장 유사한(거리가 가까운) 문서를 찾는다
# k = 3 : 가장 유사한 문서 3개를 가져오라는 뜻 
retrieved_docs = vector_store.similarity_search(query, k=3)

print(retrieved_docs)

### Retrieval & RAG (검색기 연결 및 질의응답)

In [27]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

# Retrieval 변환
# 벡터 스토어를 LangChain의 표준 'Retrieval'인터페이스로 변환 -> 나중에 Chain이나 Agent에 끼워 쓸 수 있음
retrieval = vector_store.as_retriever()

# 모델 생성
model = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0
)

system_prompt = (
    "당신은 질문, 답변을 돕는 보조원입니다. "
    "아래 제공된 context를 사용하여 질문에 답하세요. "
    "답을 모르면 모른다고 하되, 답변을 지어내지 마세요. "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# 문서 결합 체인 생성 : 역할 -> 검색된 문서들을 하나로 뭉쳐 프롬프트의 {context} 자리에 채워 넣음
combine_docs_chain = create_stuff_documents_chain(model, prompt)


# 리트리버 체인 생성 :  역할 -> 질문을 받아 검색기로 문서를 찾고, 그 문서들을 결합 체인으로 넘김
rag_chain = create_retrieval_chain(retrieval, combine_docs_chain)

response1 = rag_chain.invoke({"input": "마을 무덤에 있던 남자를 누가 죽였나요?"})

print("답변1 : ", response1['answer'])

response2 = rag_chain.invoke({"input" : "톰소여는 어떤 사람인가요?"})
print("답변2 : ", response2['answer'])


답변1 :  Injun Joe가 의사를 죽였습니다.
답변2 :  톰 소여는 모험을 사랑하는 활발한 소년입니다.  
- 집이나 학교, 친구들(Huck Finn, Joe Harper)과 함께 다양한 모험을 즐깁니다.  
- 무덤가와 동굴처럼 큰 모험도 겪습니다. 무덤가에서 Injun Joe를 보게 되어 무서워합니다.  
- 친구들을 돕고 싶어 하는 마음이 크고, Muff Potter를 돕기 위해 음식을 가져다 주는 등 친절한 면이 있습니다.  
- 때때로 장난을 치고 말썽을 부리기도 하는데, 예를 들어 책을 찢었다고 해서 교사에게 “매우 나쁜 아이”라고 들을 때가 있습니다.
